# Teaserfigure Plots

## Imports and Helper Functions

In [ ]:
import warnings
import re
import json

from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps, register_fausans_font

import biopsykit as bp
from biopsykit.stats import StatsPipeline
from biopsykit.classification.model_selection import SklearnPipelinePermuter
from biopsykit.classification.analysis import (
    predictions_as_df,
    plot_conf_matrix,
)

from stresspose_analysis.classification.utils import feature_data_long_to_wide
from stresspose_analysis.dataset import StressPoseDataset
from stresspose_analysis.data_wrangling import add_concat_feature_name_to_index

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"


palette

## Setup Paths

In [ ]:
deploy_type = "local"

In [ ]:
config_path = Path("../../config.json")
config_dict = json.load(config_path.open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
dataset = StressPoseDataset(base_path)

In [ ]:
label_mapping = {"tsst": "Stress", "ftsst": "No Stress"}

## Load Data

In [ ]:
classification_type = "general"

In [ ]:
root_path = Path("../../")
input_path = root_path.joinpath("output/classification")
output_path = root_path.joinpath("results")
img_path = output_path.joinpath("plots")

bp.utils.file_handling.mkdirs([img_path])

In [ ]:
pickle_files = sorted(input_path.glob(f"*_{classification_type}_*.pkl"))
feature_files = sorted(input_path.glob("*.csv"))

In [ ]:
index = 0

pickle_file = pickle_files[index]
feature_file = feature_files[index]

print("Selected Files:")
print(f"{pickle_file.name}")
print(f"{feature_file.name}")

### Pickled `SklearnPipelinePermuter` object

In [ ]:
pipeline_permuter = SklearnPipelinePermuter.from_pickle(pickle_file)

### Features

In [ ]:
data = bp.io.load_long_format_csv(feature_file)
data = data.rename(index=label_mapping, level="condition")
data_wide = feature_data_long_to_wide(data)

data = add_concat_feature_name_to_index(data)

data.head()

## Detailed Analysis

In [ ]:
selected_pipeline = ("VarianceThreshold", "StandardScaler", "RFE", "RandomForestClassifier")

In [ ]:
best_estimator_summary = pipeline_permuter.best_estimator_summary()
best_estimator_summary.head()

In [ ]:
predictions = predictions_as_df(pipeline_permuter, data_wide, selected_pipeline, label_mapping)
predictions.head()

### Confusion Matrix

In [ ]:
with sns.plotting_context("talk"):
    fig, ax = plt.subplots(figsize=(4, 4))
    plot_conf_matrix(predictions, labels=["Stress", "No Stress"], label_name="condition", despine=False, ax=ax)
    fig.tight_layout(pad=0.1)

    fig.savefig(img_path.joinpath("img_confusion_matrix_teaserfigure.pdf"), transparent=True)

## Motion Features

In [ ]:
steps = [("prep", "normality"), ("test", "pairwise_tests")]
params = {
    "dv": "data",
    "within": "condition",
    "subject": "subject",
    # "groupby": ["feature_type", "body_part", "channel", "type", "metric", "axis"],
    "groupby": "feature_concat",
    "multicomp": {"method": "bonf", "levels": True},
    "parametric": False,
}

pipeline = StatsPipeline(
    steps=steps,
    params=params,
)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    pipeline.apply(data);

In [ ]:
rename_map = [
    "expert-Head-ang_vel-static_periods-ratio_percent-norm",
    "expert-Trunk-ang_vel-static_periods-ratio_percent-norm",
]

In [ ]:
features_flat = {
    "sp_ratio": [
        "expert-Head-ang_vel-static_periods-ratio_percent-norm",
        "expert-Trunk-ang_vel-static_periods-ratio_percent-norm",
    ]
}

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets(
    "test", stats_effect_type="within", plot_type="multi", x="condition", subplots=False, features=features_flat
)
box_pairs

In [ ]:
data_plot = data.reindex(rename_map, level="feature_concat")
data_plot = data_plot.droplevel(data_plot.index.names[2:-1])
data_plot

In [ ]:
with sns.plotting_context("talk"):
    fig, ax = plt.subplots(figsize=(6, 4))
    rect = (0, 0, 1, 0.9)
    bp.plotting.feature_boxplot(
        data=data_plot,
        x="feature_concat",
        y="data",
        hue="condition",
        hue_order=["No Stress", "Stress"],
        legend_loc="upper center",
        legend_orientation="horizontal",
        stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues, "verbose": 0},
        palette=cmaps.faculties,
        rect=rect,
        ax=ax,
    )
    ax.set_ylim([-0.1, 1.1])
    ax.set_xticklabels(["Head", "Trunk"])
    ax.set_xlabel("Body Part")
    ax.set_yticklabels([])
    ax.set_ylabel("Amount of Freezing")
    fig.tight_layout(rect=rect)
    fig.savefig(img_path.joinpath(f"img_freezing_teaserfigure.pdf"), transparent=True)